# CREACIÓN DE NUEVAS TABLAS

# Review:

Importamos librerias

In [466]:
import pandas as pd
import uuid #es para generar ids

Cargamos la data

In [467]:
review_google = pd.read_json("C:/Users/leand/Desktop/Proyecto Final/Proyecto-Final-HENRY-Big-Data/ETL/Data florida/reviews_Florida_Google.json")

In [468]:
reviews_fl = pd.read_json("C:/Users/leand/Desktop/Proyecto Final/Proyecto-Final-HENRY-Big-Data/ETL/Data florida/yelp_reviews_fl.json")

In [469]:
reviews_fl.head(1)

,review_id,user_id,business_id,stars,useful,funny,cool,text,date
0,_ZeMknuYdlQcUqng_Im3yg,yfFzsLmaWF2d4Sr0UNbBgg,LHSTtnW3YHCeUkRDGyJOyw,5,2,0,0,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16


In [470]:
review_google.head(1)

,user_id,name,time,rating,text,gmap_id
0,1.111923e+20,Nellie Lao,1467403800000,5,Excellent Dominican/PuertoRican food. Her Arro...,0x88dae191ee505917:0x6ba3e25388d3fad4


Agregamos un id al df review_google

In [471]:
review_google['id_review_google'] = [str(uuid.uuid4()) for _ in range(len(review_google))]

Convertimos la columna time al formato correcto

In [472]:
review_google['time'] = pd.to_datetime(review_google['time'], unit='ms')
review_google['time'] = review_google['time'].dt.strftime('%Y-%m-%d %H:%M:%S')

Transformamos las columnas de los df

In [473]:
# Renombrar las columnas del segundo DataFrame para que coincidan con las del primero
review_google.rename(columns={'time': 'date', 'id_review_google': 'review_id', 'rating': 'stars'}, inplace=True)

# Agregar una columna 'id_restaurante' con valores nulos para ambos DataFrames
reviews_fl['id_restaurante'] = reviews_fl['business_id']
review_google['id_restaurante'] = review_google['gmap_id']

# Agregar una columna 'id_usuario' para ambos DataFrames
reviews_fl['id_usuario'] = reviews_fl['user_id']
review_google['id_usuario'] = review_google['user_id']

# Agregar una columna 'id_plataforma' para ambos DataFrames
reviews_fl['id_plataforma'] = 'Yelp'
review_google['id_plataforma'] = 'Google'

Unimos los df

In [474]:
# Unir ambos DataFrames
df_combined = pd.concat([reviews_fl, review_google], ignore_index=True)

Creamos un nuveo id a la columna review_id

In [475]:
df_combined['review_id'] = range(len(df_combined))

Elegimos las columnas que vamos a querer para el nuevo df

In [476]:
# Seleccionar las columnas requeridas
df_reviews = df_combined[['review_id', 'id_restaurante', 'id_usuario', 'id_plataforma',
                          'stars', 'text', 'date', 'useful', 'funny', 'cool']]

Renombramos algunas columnas

In [477]:
# Renombrar las columnas del segundo DataFrame para que coincidan con las del primero
df_reviews.rename(columns={'stars': 'raiting', 'review_id': 'id_review', 'text': 'review'}, inplace=True)

C:\Users\leand\AppData\Local\Temp\ipykernel_11192\2036404310.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_reviews.rename(columns={'stars': 'raiting', 'review_id': 'id_review', 'text': 'review'}, inplace=True)


In [478]:
# Mostrar el nuevo DataFrame
df_reviews.head(1)

,id_review,id_restaurante,id_usuario,id_plataforma,raiting,review,date,useful,funny,cool
0,0,LHSTtnW3YHCeUkRDGyJOyw,yfFzsLmaWF2d4Sr0UNbBgg,Yelp,5,Amazingly amazing wings and homemade bleu chee...,2015-08-07 02:29:16,2.0,0.0,0.0


# Cargamos df restaurantes


Para actualizar el id y que coincida con las otras tablas

In [479]:
import json

# Lee el archivo JSON como texto
with open("C:/Users/leand/Desktop/Proyecto Final/Proyecto-Final-HENRY-Big-Data/Tablas/restaurantes.json", "r") as file:
    json_text = file.read()

# Divide el texto en objetos JSON individuales
json_objects = json_text.split("\n")

# Convierte los objetos JSON en una lista de diccionarios
data_list = [json.loads(obj) for obj in json_objects if obj.strip()]

# Crea un DataFrame a partir de la lista de diccionarios
restaurantes = pd.DataFrame(data_list)

In [480]:
restaurantes.head(1)

,id_restaurante,id_ciudad,identificador_yelp,identificador_google,nombre,direccion,latitud,longitud,avg_rating,reviews_count,estado
0,0,51,None,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1,Cape Seafood Shack,603 Del Prado Blvd S,26.641377,-81.940545,5.0,1,Normally open


Nos quedamos solo con las columnas que nos interesan

In [481]:
# Selecciona solo las columnas deseadas
columnas_deseadas = ['id_restaurante', 'identificador_yelp', 'identificador_google']
restaurantes_filtrado = restaurantes[columnas_deseadas].copy()

In [482]:
restaurantes_filtrado.head(1)

,id_restaurante,identificador_yelp,identificador_google
0,0,None,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1


Combinamos en una sola columna y luego eliminamos las otras

In [483]:
# Combina las columnas 'identificador_yelp' e 'identificador_google'
restaurantes_filtrado['identificador_plataforma'] = restaurantes_filtrado['identificador_yelp'].combine_first(restaurantes_filtrado['identificador_google'])

# Elimina las columnas originales 'identificador_yelp' e 'identificador_google'
restaurantes_filtrado = restaurantes_filtrado.drop(['identificador_yelp', 'identificador_google'], axis=1)


Renombramos asi coinciden con el otro df

In [484]:
restaurantes_filtrado.rename(columns={'id_restaurante': 'id', 'identificador_plataforma': 'id_restaurante'}, inplace=True)

In [485]:
restaurantes_filtrado.head()

,id,id_restaurante
0,0,0x88db4147b1d9e6f3:0x943dbd10a92ba1b1
1,1,0x88c2e4e34f1ed783:0x76c5da381c499d79
2,2,0x88dae191ee505917:0x6ba3e25388d3fad4
3,3,0x88d9c754413f6c9d:0x1f93eff5e0ba9c16
4,4,0x88d9b99475d9fd7b:0xea6083d207b2471a


Unimos los df

In [486]:
reviews = pd.merge(restaurantes_filtrado, df_reviews, on='id_restaurante', how='inner')

In [487]:
reviews.head(2)


,id,id_restaurante,id_review,id_usuario,id_plataforma,raiting,review,date,useful,funny,cool
0,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895976,111192300499999997952.0,Google,5,Excellent Dominican/PuertoRican food. Her Arro...,2016-07-01 20:10:00,NaN,NaN,NaN
1,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895977,108990109100000002048.0,Google,5,The food and service is great . In my top 3 do...,2016-08-17 07:42:00,NaN,NaN,NaN


Les devolvemos sus nombres originales

In [488]:
reviews.rename(columns={'id_restaurante': 'identificador_plataforma', 'id': 'id_restaurante'}, inplace=True)

In [489]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1892477 entries, 0 to 1892476
Data columns (total 11 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   id_restaurante            int64  
 1   identificador_plataforma  object 
 2   id_review                 int64  
 3   id_usuario                object 
 4   id_plataforma             object 
 5   raiting                   int64  
 6   review                    object 
 7   date                      object 
 8   useful                    float64
 9   funny                     float64
 10  cool                      float64
dtypes: float64(3), int64(3), object(5)
memory usage: 158.8+ MB


# Ahora lo mismo con el df de usuarios

In [490]:
usuarios = pd.read_csv('C:/Users/leand/Desktop/Proyecto Final/Proyecto-Final-HENRY-Big-Data/Tablas/users.csv')

In [491]:
usuarios.tail(2)

,user_identifier,user_id,platform_id,name,avg_stars,review_count,fans_count,compliment_count,cool_count,funny_count
814044,1x3KMskYxOuJCjRz70xOqQ,9231110320,1,Shomari,NaN,4.0,0.0,0.0,0.0,1.0
814045,wL5jPrLRVCK_Pmo4lM1zpA,9231110384,1,Isa,NaN,2.0,0.0,0.0,0.0,0.0


Dejamos las columnas necesarias

In [492]:
# Selecciona solo las columnas deseadas
columnas_deseadas = ['user_identifier', 'user_id']
usuarios = usuarios[columnas_deseadas]

Cambiamos los nombres asi coinciden

In [493]:
usuarios.rename(columns={'user_identifier': 'id_usuario', 'user_id': 'id'}, inplace=True)

In [494]:
reviews = pd.merge(reviews,usuarios, on='id_usuario', how='left')

In [495]:
reviews.head(5)

,id_restaurante,identificador_plataforma,id_review,id_usuario,id_plataforma,raiting,review,date,useful,funny,cool,id
0,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895976,111192300499999997952.0,Google,5,Excellent Dominican/PuertoRican food. Her Arro...,2016-07-01 20:10:00,NaN,NaN,NaN,NaN
1,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895977,108990109100000002048.0,Google,5,The food and service is great . In my top 3 do...,2016-08-17 07:42:00,NaN,NaN,NaN,NaN
2,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895978,112239077299999997952.0,Google,5,Amazing food!,2017-05-09 12:34:00,NaN,NaN,NaN,NaN
3,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895979,114435073999999991808.0,Google,5,Amazing food!,2017-04-23 17:03:00,NaN,NaN,NaN,NaN
4,2,0x88dae191ee505917:0x6ba3e25388d3fad4,895980,112048916199999995904.0,Google,5,Execellent Dominican food!!!,2014-07-05 15:58:00,NaN,NaN,NaN,NaN


In [496]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1892477 entries, 0 to 1892476
Data columns (total 12 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   id_restaurante            int64  
 1   identificador_plataforma  object 
 2   id_review                 int64  
 3   id_usuario                object 
 4   id_plataforma             object 
 5   raiting                   int64  
 6   review                    object 
 7   date                      object 
 8   useful                    float64
 9   funny                     float64
 10  cool                      float64
 11  id                        float64
dtypes: float64(4), int64(3), object(5)
memory usage: 173.3+ MB


En caso de que el id quede nulo le ponemos el id original

In [497]:
reviews['id'].fillna(reviews['id_usuario'], inplace=True)

C:\Users\leand\AppData\Local\Temp\ipykernel_11192\2405384056.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[1.111923005e+20 1.089901091e+20 1.122390773e+20 ...
 '1zEwvAm5TqJh7eqYShNVQA' 'YHceEmeft2GZXpGrVApXQQ'
 '-dQAzxnGUlRYI4TMk7L7JQ']' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  reviews['id'].fillna(reviews['id_usuario'], inplace=True)


In [498]:
reviews = reviews.drop('id_usuario', axis=1)

In [499]:
reviews.rename(columns={'id': 'id_usuario'}, inplace=True)

In [500]:
reviews.tail(1)

,id_restaurante,identificador_plataforma,id_review,id_plataforma,raiting,review,date,useful,funny,cool,id_usuario
1892476,26610,esBGrrmuZzSiECyRBoKvvA,895634,Yelp,5,The pizza was thin crust and light. Great spot...,2021-11-14 16:53:07,1.0,0.0,0.0,9230124624.0


In [502]:
reviews.info(1)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1892477 entries, 0 to 1892476
Data columns (total 11 columns):
 #   Column                    Dtype  
---  ------                    -----  
 0   id_restaurante            int64  
 1   identificador_plataforma  object 
 2   id_review                 int64  
 3   id_plataforma             object 
 4   raiting                   int64  
 5   review                    object 
 6   date                      object 
 7   useful                    float64
 8   funny                     float64
 9   cool                      float64
 10  id_usuario                object 
dtypes: float64(3), int64(3), object(5)
memory usage: 158.8+ MB


Exportamos el df

In [503]:
'''
reviews.to_csv('reviews.csv', index=False)
'''